In [1]:
import pandas as pd
import numpy as np
import itertools
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.mixture import GaussianMixture

In [2]:
PATH = "./data/Facebook_metrics/dataset_Facebook.csv"
SEPARATOR = ";"
DUMMY_COLUMNS = [
    "Type"
]

In [3]:
# metric functions
# --------------------------------------------
def total_sum_of_squares_df(df, centroid = None):
    """ Calculates and returns the TTS of the given DataFrame """
    if centroid is None:
        centroid = find_centroid_df(df)
        
    return total_sum_of_squares(df.as_matrix(), centroid)

def total_sum_of_squares(data, centroid):
    """ Calculates and returns the TTS of the given matrix
    
    Arguments:
      data - Iterable<Iterable>
      centroid - Array
    """        
    total = 0
    
    for row in data:
        for index, value in enumerate(row):
            diff = value - centroid[index]
            diffsq = diff ** 2
            total += diffsq
            
    return total


def find_centroid_df(df):
    """ Calculates and returns the centroid for a DataFrame """
    return df.mean()

# clustering functions
# --------------------------

def get_cluster_indexes(cluster_assignments):
    cluster_slices = {}
    
    for index, assignment in enumerate(assignments):
        if assignment not in cluster_slices:
            cluster_slices[assignment] = list()
            
        cluster_slices[assignment].append(index)
        
    return cluster_slices

def get_cluster_data(df, assignments):
    cluster_indexes = get_cluster_indexes(assignments)
    
    cluster_data = {k: df.iloc[v] for k, v in cluster_indexes.items()}
    
    return cluster_data

def get_clusters(df, assignments):
    """Returns an array of tuples with (<cluster>, <cluster_centroid>, <cluster_data_points>)"""
    
    return [
        (cluster, find_centroid_df(cluster_data), cluster_data) 
        for cluster, cluster_data 
        in get_cluster_data(df, assignments).items()
    ]
    

# model runners
# ------------------------------------------

def run_gaussian_mixture(model, data):
    model.fit(data)
    return model.predict(data)

def run_kmeans(model, data):
    model.fit(data)
    return model.predict(data)

def run_hclustering(model, data):
    return model.fit_predict(data)

# data functions 
# -------------------------------------------

def clean_data(data):
    # Strip whitespaces from all string values
    # and replace "?" with None,
    # and drop all na rows
    data = data.apply(lambda x: x.str.strip() if x.dtype == "object" else x) \
               .replace(["?"], [None]) \
               .dropna()

    return data

def prepare_data(data):
    return pd.get_dummies(data, columns=DUMMY_COLUMNS)

def read_data(path):
    dataset = pd.read_csv(path, sep=SEPARATOR, header=0)
    dataset = clean_data(dataset)
    dataset = prepare_data(dataset)
    return dataset

In [4]:
# Reading the dataset
# df = pd.DataFrame({
#    "A": [1,2,2,1],
#    "B": [0,0,0,1]
# })

df = read_data(PATH)

In [7]:
tss = total_sum_of_squares_df(df)
print("tss for data = %s" % tss)

tts for data = 5.14204328974e+12


In [8]:
models = [
    ("KMeans", lambda k: KMeans(n_clusters=k), run_kmeans),
    ("H-Clustering", lambda k: AgglomerativeClustering(n_clusters=k), run_hclustering),
    ("Gaussian Mixture", lambda k: GaussianMixture(n_components=k, reg_covar=0.001), run_gaussian_mixture)
]

In [10]:
for model_name, create_model, run_model in models:
    print("-------------------------------")
    print(model_name)
    print("-------------------------------")
    print("")
    for k in range(1,11):
        print("Calculating %s clusters..." % k)
        print("")
        model = create_model(k)
        assignments = run_model(model, df)
        clusters = get_clusters(df, assignments)

        twss = 0
        for cluster, centroid, cluster_slice in clusters:
            cluster_tss = total_sum_of_squares_df(cluster_slice, centroid)
            print("cluster %s | tss = %s | size = %s" % (cluster, cluster_tss, len(cluster_slice)))
            twss += cluster_tts

        print("twss/tss = %s/%s = %s" % (twss, tss, twss / tss))
        print("")

-------------------------------
Gaussian Mixture
-------------------------------

Calculating 1 clusters...

cluster 0 | tts = 5.14204328974e+12 | size = 495
ttws/tts = 5.14204328974e+12/5.14204328974e+12 = 1.0

Calculating 2 clusters...

cluster 0 | tts = 1.9613809649e+12 | size = 493
cluster 1 | tts = 204331351308.0 | size = 2
ttws/tts = 2.1657123162e+12/5.14204328974e+12 = 0.421177379141

Calculating 3 clusters...

cluster 1 | tts = 149723615348.0 | size = 376
cluster 0 | tts = 1.17348930384e+12 | size = 117
cluster 2 | tts = 204331351308.0 | size = 2
ttws/tts = 1.52754427049e+12/5.14204328974e+12 = 0.297069508058

Calculating 4 clusters...

cluster 0 | tts = 136808856235.0 | size = 368
cluster 3 | tts = 534247589838.0 | size = 120
cluster 2 | tts = 388641769413.0 | size = 6
cluster 1 | tts = 0.0 | size = 1
ttws/tts = 1.05969821549e+12/5.14204328974e+12 = 0.206085043586

Calculating 5 clusters...

cluster 0 | tts = 128757475660.0 | size = 365
cluster 4 | tts = 280369644965.0 | size 

cluster 3 | tts = 15952762235.1 | size = 231
cluster 7 | tts = 19558111361.3 | size = 88
cluster 0 | tts = 91968220687.2 | size = 53
cluster 1 | tts = 65942760935.0 | size = 10
cluster 2 | tts = 37055750258.8 | size = 108
cluster 4 | tts = 0.0 | size = 1
cluster 6 | tts = 5280858658.67 | size = 3
cluster 5 | tts = 0.0 | size = 1
ttws/tts = 235758464136.0/5.14204328974e+12 = 0.0458491791787

Calculating 9 clusters...

cluster 1 | tts = 15952762235.1 | size = 231
cluster 7 | tts = 19558111361.3 | size = 88
cluster 8 | tts = 24675835427.2 | size = 37
cluster 3 | tts = 21379736032.6 | size = 16
cluster 0 | tts = 65942760935.0 | size = 10
cluster 2 | tts = 37055750258.8 | size = 108
cluster 4 | tts = 0.0 | size = 1
cluster 6 | tts = 5280858658.67 | size = 3
cluster 5 | tts = 0.0 | size = 1
ttws/tts = 189845814909.0/5.14204328974e+12 = 0.0369203066197

Calculating 10 clusters...

cluster 1 | tts = 15952762235.1 | size = 231
cluster 7 | tts = 19558111361.3 | size = 88
cluster 8 | tts = 246758